<a href="https://colab.research.google.com/github/carloslme/data-engineering/blob/main/get_and_save_list_objects_s3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This notebook has the code to perform a bucket validation after a migration from AWS S3 to an endpoint, for this case, Wasabisys (also called Wasabi).

# Contents
* Prerequisites
* Step 1: Import libraries
* Step 2: Setting credentials
* Step 3: Declare functions
* Step 4: Validate S3 objects in the Wasabi bucket
